## Fake News Classificer USsing Bidirectional LSTM

 Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv download and upload to google colab

Google Colab [Link](https://colab.research.google.com/drive/1pr62TULDLlW5U7CfP9cdAXcnG1LfkzD9?authuser=5#scrollTo=1LyqhJFUGKdP)

In [163]:
import pandas as pd

In [164]:
df=pd.read_csv('train.csv',engine='python',error_bad_lines=False)

<ipython-input-164-772ef6e293b2>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('train.csv',engine='python',error_bad_lines=False)
Skipping line 10723: unexpected end of data


In [165]:
df.isnull().sum()

id           0
title      292
author    1017
text        22
label        0
dtype: int64

In [166]:
df.shape

(10721, 5)

In [167]:
# remove null value
df=df.dropna()

In [168]:
df.shape

(9412, 5)

In [169]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [170]:
## get the independent features
X=df.drop('label',axis=1)

In [171]:
y=df['label']

In [172]:
## Check whether dataset is balanced or not
y.value_counts()

0    5335
1    4077
Name: label, dtype: int64

In [173]:
import tensorflow as tf
tf.__version__

'2.15.0'

In [174]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional #for bi directional RNN

In [175]:
## vocabulary size
voc_size=5000

In [176]:
messages=X.copy()

In [177]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [178]:
import nltk
import re
from nltk.corpus import stopwords

In [179]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [180]:
messages.reset_index(inplace=True)

In [181]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [182]:
corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [183]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr[:5]

[[459, 1072, 941, 2898, 3338, 3337, 3935, 1118, 3654, 290],
 [3325, 3338, 593, 582, 1166, 3520, 4998],
 [3935, 4952, 4975, 355],
 [3418, 2663, 3940, 2155, 4462, 4179],
 [3794, 1166, 4542, 1859, 58, 2772, 1166, 3463, 4616, 3268]]

## Embedding Representation

In [184]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 1118, 3654,  290],
       [   0,    0,    0, ..., 1166, 3520, 4998],
       [   0,    0,    0, ..., 4952, 4975,  355],
       ...,
       [   0,    0,    0, ..., 3317, 3069, 4013],
       [   0,    0,    0, ..., 3536,  153, 4998],
       [   0,    0,    0, ..., 3775, 3408, 4998]], dtype=int32)

## Creating Model

In [185]:
## Creating model
embedding_vector_features=40 # each word will have 40 dimension of vector
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_5 (Bidirecti  (None, 200)               112800    
 onal)                                                           
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [186]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [187]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [188]:
print(X_train.shape)
print(y_train.shape)

(6306, 20)
(6306,)


## Model Training

In [189]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
198/198 [==============================] - 12s 42ms/step - loss: 0.3179 - accuracy: 0.8536 - val_loss: 0.2090 - val_accuracy: 0.9156
Epoch 2/10
198/198 [==============================] - 7s 35ms/step - loss: 0.1315 - accuracy: 0.9515 - val_loss: 0.2277 - val_accuracy: 0.9070
Epoch 3/10
198/198 [==============================] - 7s 36ms/step - loss: 0.0684 - accuracy: 0.9780 - val_loss: 0.3031 - val_accuracy: 0.9108
Epoch 4/10
198/198 [==============================] - 8s 40ms/step - loss: 0.0514 - accuracy: 0.9833 - val_loss: 0.3772 - val_accuracy: 0.8918
Epoch 5/10
198/198 [==============================] - 7s 34ms/step - loss: 0.0270 - accuracy: 0.9921 - val_loss: 0.3944 - val_accuracy: 0.8995
Epoch 6/10
198/198 [==============================] - 7s 36ms/step - loss: 0.0129 - accuracy: 0.9976 - val_loss: 0.4295 - val_accuracy: 0.8970
Epoch 7/10
198/198 [==============================] - 6s 31ms/step - loss: 0.0109 - accuracy: 0.9973 - val_loss: 0.4397 - val_accuracy: 0.898

In [190]:
y_pred=model.predict(X_test)

98/98 [==============================] - 2s 8ms/step


In [191]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

## Model Performance

In [192]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[1631,  159],
       [ 172, 1144]])

In [193]:
print(accuracy_score(y_test,y_pred))

0.8934320669671604


In [194]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.91      0.91      1790
           1       0.88      0.87      0.87      1316

    accuracy                           0.89      3106
   macro avg       0.89      0.89      0.89      3106
weighted avg       0.89      0.89      0.89      3106

